In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [10]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jan_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [15]:
x[:,:2]

matrix([[ 2393.97041023,  2374.81086685],
        [ 2343.05589721,  2314.46109355],
        [ 2892.07425476,  2851.10871439],
        [ 2873.36629234,  2894.618169  ],
        [ 2934.19661288,  2913.60174191],
        [ 2786.56684629,  2722.42462337],
        [ 2457.23628061,  2320.32301718],
        [ 2342.04884456,  2373.39056068],
        [ 3027.69148029,  2967.391873  ],
        [ 2873.83301966,  2972.54077379],
        [ 2726.25778847,  2567.16556557],
        [ 2357.60761309,  2444.98962748],
        [ 2166.91537171,  2318.94792705],
        [ 2466.07520272,  2401.38482722],
        [ 2390.74913504,  2407.18381175],
        [ 1652.07837092,  1908.5450188 ],
        [ 1717.2936292 ,  2350.80981197],
        [ 2322.3774848 ,  2392.11122478],
        [ 2516.00793045,  2557.20848872],
        [ 2441.02809097,  2551.05261178],
        [ 2567.70737889,  2531.83760797],
        [ 2552.07785406,  2551.26332641],
        [ 2329.28614702,  2389.43328596],
        [ 2377.8692257 ,  2432.635

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e+00, 3e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 26
 AA' NZ     : 3.250e+02
 Factor NZ  : 3.510e+02
 Factor Ops : 6.201e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.83411251e+07  0.00000000e+00  6.44e+03 2.16e+03  1.00e+06     0s
   1  -1.56496792e+07 -9.41431304e+06  1.68e+03 5.64e+02  3.50e+05     0s
   2  -1.20957178e+07 -1.36828328e+07  4.67e+01 1.57e+01  4.03e+04     0s
   3  -1.22583670e+07 -1.24434308e+07  6.25e-01 2.10e-01  3.4

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.2067420564290672e-13,
 1780.5724665886473,
 1.3246051314502833e-13,
 2849.4343291572927,
 1.5628178413220782e-05,
 3.869548771453981e-05,
 3.9972433535125596e-11,
 1245.8315785009806,
 43.77837271067938,
 2340.5386336108895,
 2.8077026226620318e-05,
 8.828465888844825e-05,
 3.868572212851595e-05,
 3.891223752047063e-05,
 2017.232504240645,
 12.765154636031326,
 2.318466377292626e-05,
 2.309694099990019e-05,
 2232.0560010942277,
 3.869197488303583e-05,
 3.869886633212343e-05,
 2.815740171503169e-06,
 2543.094382025593,
 0.00186850266826101,
 0.023182280451389925,
 0.010911558477525396,
 1871.1218171070168,
 2308.690768953434,
 1691.8223110358829,
 1234.1898339929007,
 0.0036254740848237765,
 730.1950365628765,
 1876.6506878898008,
 55.004566079254296,
 150.46226573764977,
 0.18129024914077263,
 297.82724253160256,
 1807.3718559785334,
 1469.9750470440827,
 0.006001414206789846,
 2399.4136810984105,
 835.0087000897648,
 2.4497922260140144e-07,
 2.449813633885455e-07,
 2.44980092883763